<a href="https://colab.research.google.com/github/ORCT/ds-section1-project/blob/main/Q0_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload();

In [2]:
import pandas as pd

df = pd.read_csv('vgames2.csv',index_col=0)#index_col=0로 unnamed0 column 삭제
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [3]:
df.info();#year, genre, publisher 칼럼에 결측치 존재

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB


In [4]:
year_null = df[df['Year'].isnull()]
genre_null = df[df['Genre'].isnull()]
pub_null = df[df['Publisher'].isnull()]

In [5]:
!pip install wikipedia-api;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import sys
sys.path.append('/content/wiki_crawler.py')

import wiki_crawler

In [7]:
wc = wiki_crawler.WikiCrawler();

In [ ]:
year_null = wc.fill_all(year_null);#찾은거 193, 실패 73
genre_null = wc.fill_all(genre_null);#찾은거 32, 실패 18
pub_null = wc.fill_all(pub_null);#찾은거 10, 실패 48

In [9]:
year_null.shape#결측치 다 채우고 나서 새로 csv 파일로 저장할 것

(271, 9)

In [10]:
df_null = df[(df['Year'].isnull()) | (df['Genre'].isnull()) | (df['Publisher'].isnull())]#pub_unknown을 포함하면 데이터가 꽤 늘어나는데(459행) 공동 출자처럼 퍼블리셔가 여러 기업일 경우 저렇게 나오더라 물론 내가 몇 개 채워서 조금 줄긴 했을 거임
df_null.shape

(357, 9)

In [11]:
for i in genre_null.index:
  df.loc[i,'Genre'] = genre_null.loc[i,'Genre']
for i in year_null.index:
  df.loc[i, 'Year'] = year_null.loc[i, 'Year']
for i in pub_null.index:
  df.loc[i, 'Publisher'] = pub_null.loc[i, 'Publisher']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16522 non-null  float64
 3   Genre        16580 non-null  object 
 4   Publisher    16549 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.8+ MB


In [12]:
# Sales  
sales_col = df.columns[5:]

# K인 곳의 인덱스 저장
loc_k = df[df.NA_Sales.str.contains('K')].index

for col in sales_col:
    # k인 데이터 위치 가져오기
    na_k = df.loc[df[col].str.contains('K'), col]
    # m인 데이터 위치 가져오기
    na_m = df.loc[df[col].str.contains('M'), col]
    # 1000으로 나누고 K빼고 저장
    na_k = na_k.str.replace('K', '').astype(float) / 1000
    # M 빼고 저장
    na_m = na_m.str.replace('M', '').astype(float)
    
    # 기존 type이 string이기 때문에 변경해서 저장
    df.loc[df[col].str.contains('K'), col] = na_k.astype(str)
    df.loc[df[col].str.contains('M'), col] = na_m.astype(str)
df.loc[loc_k]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
11,Ford Racing,PS,2001.0,Racing,Empire Interactive,0.48,0.33,0.0,0.06
46,Resident Evil: Revelations 2,PS3,2015.0,Action,Capcom,0.06,0.08,0.16,0.03
146,TOCA Race Driver 3,PC,2006.0,Racing,Codemasters,0.0,0.02,0,0
879,Tears to Tiara Anecdotes: The Secret of Avalon,PS3,2009.0,Strategy,Aqua Plus,0.0,0,0.04,0
1087,Pro Evolution Soccer 2014,3DS,2013.0,Action,Konami Digital Entertainment,0.0,0,0.19,0
...,...,...,...,...,...,...,...,...,...
15648,Championship Manager 2010,PC,2009.0,Sports,Eidos Interactive,0.0,0.01,0,0
16195,Vin Diesel: Wheelman,PS3,2009.0,Racing,Ubisoft,0.12,0.15,0,0.06
16331,Midway Arcade Treasures,PS2,2003.0,Misc,Midway Games,0.72,0.56,0,0.19
16493,Mortal Kombat,PSV,2012.0,Fighting,Warner Bros. Interactive Entertainment,0.47,0.18,0,0.12


In [13]:
col = df.columns[-4:]
# 실수형으로 변환
df[col] = df[col].astype(float)
df.dtypes

Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
dtype: object

In [14]:
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04
...,...,...,...,...,...,...,...,...,...
16594,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01
16595,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00
16596,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16597,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [15]:
df.loc[1,'NA_Sales':'Other_Sales'].sum()

0.04

In [16]:
for i in df.index:
  df.loc[i,'Global_Sales'] = df.loc[i,'NA_Sales':'Other_Sales'].sum()

In [17]:
#글로벌 판매량이 높지 않으면서 결측치를 가지고 있는 행을 삭제하면 좋을 것 같은데
df_drop_year = df[(df['Year'].isnull()) & (df['Global_Sales']>1)]
df_drop_genre = df[(df['Genre'].isnull()) & (df['Global_Sales']>1)]
df_drop_pub = df[(df['Publisher'].isnull()) & (df['Global_Sales']>1)]

In [18]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16471 entries, 1 to 16598
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16471 non-null  object 
 1   Platform      16471 non-null  object 
 2   Year          16471 non-null  float64
 3   Genre         16471 non-null  object 
 4   Publisher     16471 non-null  object 
 5   NA_Sales      16471 non-null  float64
 6   EU_Sales      16471 non-null  float64
 7   JP_Sales      16471 non-null  float64
 8   Other_Sales   16471 non-null  float64
 9   Global_Sales  16471 non-null  float64
dtypes: float64(6), object(4)
memory usage: 1.4+ MB


In [19]:
df.reset_index(drop=True,inplace=True)
df#데이터 결측치 메꾸기 완료

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00,0.04
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,0.18
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00,0.02
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00,0.04
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04,0.25
...,...,...,...,...,...,...,...,...,...,...
16466,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01,0.20
16467,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00,0.01
16468,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13,0.79
16469,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03,0.38


In [20]:
# 년도 데이터 이상수치 확인 및 담아두기
year = df.Year.unique()
temp = []
for i in year:
    if int(i) <= 1900:
        print(i)
        temp.append(i)

9.0
97.0
11.0
15.0
3.0
96.0
6.0
8.0
1.0
5.0
4.0
10.0
98.0
7.0
16.0
86.0
14.0
95.0
2.0
13.0
0.0
12.0
94.0


In [21]:
df['Year'] = df['Year'].astype(int)
# 데이터 수치 통일화를 위해서 연도 데이터 변경
df.loc[(df.Year.values <= 80), 'Year'] += 2000
df.loc[((df.Year.values >= 80) & (df.Year.values <= 100)), 'Year'] += 1900

df.Year.value_counts()

2008    1447
2009    1432
2010    1278
2007    1211
2011    1159
2006    1023
2005     948
2002     845
2003     786
2004     754
2012     658
2015     613
2014     580
2013     548
2001     490
1998     380
2000     354
2016     342
1999     342
1997     291
1996     264
1995     219
1994     121
1993      60
1981      46
1992      43
1991      41
1982      37
1986      21
1989      19
1983      17
1988      17
1987      16
1990      16
1984      15
1985      14
1980      10
1978       5
1977       3
2017       3
2020       1
2021       1
1979       1
Name: Year, dtype: int64

In [22]:
df.to_csv('vgames_pre.csv')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16471 entries, 0 to 16470
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16471 non-null  object 
 1   Platform      16471 non-null  object 
 2   Year          16471 non-null  int64  
 3   Genre         16471 non-null  object 
 4   Publisher     16471 non-null  object 
 5   NA_Sales      16471 non-null  float64
 6   EU_Sales      16471 non-null  float64
 7   JP_Sales      16471 non-null  float64
 8   Other_Sales   16471 non-null  float64
 9   Global_Sales  16471 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 1.3+ MB
